In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from collections import namedtuple
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import torch
import wandb

from src.data.filesystem import fopen
from src.data.utils import load_train_test
from src.eval import metrics
from src.models.swivel import get_swivel_embeddings, get_best_swivel_matches
from src.models.swivel_encoder import SwivelEncoderModel, convert_names_to_model_inputs, train_swivel_encoder

In [ ]:
# Config

given_surname = "given"
size = "freq"
vocab_size = 500000
embed_dim = 200

Config = namedtuple("Config", "train_path test_path embed_dim n_layers char_embed_dim n_hidden_units bidirectional lr batch_size use_adam_opt n_epochs swivel_vocab_path swivel_model_path encoder_model_path")
config = Config(
    train_path=f"s3://familysearch-names/processed/tree-hr-{given_surname}-similar-train-{size}.csv.gz",
    test_path=f"s3://familysearch-names/processed/tree-hr-{given_surname}-similar-test-{size}.csv.gz",
    embed_dim=embed_dim,
    n_layers = 3,
    char_embed_dim = 32,
    n_hidden_units = 400,
    bidirectional = True,
    lr = 0.03,
    batch_size=128,
    use_adam_opt = False,
    n_epochs=200,
    swivel_vocab_path=f"s3://nama-data/data/models/fs-{given_surname}-{size}-swivel-vocab-{vocab_size}.csv",
    swivel_model_path=f"s3://nama-data/data/models/fs-{given_surname}-{size}-swivel-model-{vocab_size}-{embed_dim}.pt",
    encoder_model_path=f"s3://nama-data/data/models/fs-{given_surname}-{size}-encoder-model-{vocab_size}-{embed_dim}.pt",
)

In [ ]:
wandb.init(
    project="nama",
    entity="nama",
    name="56_swivel_encoder",
    group=given_surname,
    notes="",
    config=config._asdict()
)

### Load data

In [ ]:
[train, test] = load_train_test([config.train_path, config.test_path])

In [ ]:
input_names_train, weighted_actual_names_train, candidate_names_train = train
input_names_test, weighted_actual_names_test, candidate_names_test = test

In [ ]:
print("input_names_train", len(input_names_train))
print("weighted_actual_names_train", sum(len(wan) for wan in weighted_actual_names_train))
print("candidate_names_train", len(candidate_names_train))

print("input_names_test", len(input_names_test))
print("weighted_actual_names_test", sum(len(wan) for wan in weighted_actual_names_test))
print("candidate_names_test", len(candidate_names_test))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
swivel_vocab_df = pd.read_csv(fopen(config.swivel_vocab_path, "rb"))
print(swivel_vocab_df.head(5))

In [ ]:
swivel_vocab = {name: _id for name, _id in zip(swivel_vocab_df["name"], swivel_vocab_df["index"])}
print(swivel_vocab["<john>"])

In [ ]:
swivel_model = torch.load(fopen(config.swivel_model_path, "rb"))
print(swivel_model)

### Train

In [ ]:
# train using all names in the vocabulary
train_names = swivel_vocab.keys()

In [ ]:
train_embeddings = torch.Tensor(get_swivel_embeddings(swivel_model, swivel_vocab, train_names))
print(train_embeddings.shape)

In [ ]:
train_model_inputs = convert_names_to_model_inputs(train_names)
print(train_model_inputs.shape)
print(train_model_inputs.dtype)

In [ ]:
# create model
encoder_model = SwivelEncoderModel(n_layers=config.n_layers,
                                   char_embed_dim=config.char_embed_dim,
                                   n_hidden_units=config.n_hidden_units,
                                   output_dim=config.embed_dim,
                                   bidirectional=config.bidirectional,
                                   pack=False,
                                   device=device)

In [ ]:
train_swivel_encoder(encoder_model,
                     train_model_inputs,
                     train_embeddings,
                     num_epochs=config.n_epochs,
                     batch_size=config.batch_size,
                     lr=config.lr,
                     use_adam_opt=config.use_adam_opt,
                     use_mse_loss=False)

### Eval

In [ ]:
# get best matches
# NOTE: only considers as potential matches names in candidate_names_test, not names in input_names_test
k = 100
batch_size = 256
add_context = True
n_jobs=4
best_matches = get_best_swivel_matches(swivel_model,
                                       swivel_vocab,
                                       input_names_test,
                                       candidate_names_test,
                                       k,
                                       batch_size,
                                       add_context=add_context,
                                       encoder_model=encoder_model,
                                       n_jobs=n_jobs)

#### PR Curve

In [ ]:
metrics.precision_weighted_recall_curve_at_threshold(
    weighted_actual_names_test, best_matches, min_threshold=0.01, max_threshold=2.0, step=0.05, distances=False
)

#### AUC

In [ ]:
print(metrics.get_auc(
    weighted_actual_names_test, best_matches, min_threshold=0.01, max_threshold=2.0, step=0.05, distances=False
))

In [ ]:
wandb.finish()

### Test

In [ ]:
test_names = ["<john>", "<johnny>", "<jonathan>",
              "<mary>", "<marie>", "<maria>"]
test_embeddings = torch.Tensor(get_swivel_embeddings(swivel_model, swivel_vocab, test_names))
print(test_embeddings.shape)

In [ ]:
print(test_names[0:3])
print(cosine_similarity(test_embeddings[0:1], test_embeddings[0:3]))
print(test_names[3:])
print(cosine_similarity(test_embeddings[0:1], test_embeddings[3:]))

In [ ]:
test_model_inputs = convert_names_to_model_inputs(test_names)
print(test_model_inputs.shape)
print(test_model_inputs.dtype)

In [ ]:
# create model
n_layers = 1
char_embed_dim = 64
n_hidden_units = 200
embed_dim = 200
bidirectional = True
pack = False
encoder_model = SwivelEncoderModel(n_layers=n_layers, char_embed_dim=char_embed_dim, n_hidden_units=n_hidden_units,
                                   output_dim=embed_dim, bidirectional=bidirectional, pack=pack, device=device)

In [ ]:
lr = 0.01
n_epochs=100
use_adam_opt = False
use_mse_loss = False
train_swivel_encoder(encoder_model, test_model_inputs, test_embeddings, num_epochs=n_epochs, batch_size=64, lr=lr,
                     use_adam_opt=use_adam_opt, use_mse_loss=use_mse_loss)

In [ ]:
test_embeddings_predicted = encoder_model(test_model_inputs).detach().cpu().numpy()

In [ ]:
test_embeddings_numpy = test_embeddings.cpu().numpy()

In [ ]:
cosine_similarity(test_embeddings_numpy, test_embeddings_predicted)

In [ ]:
cosine_similarity(test_embeddings_numpy, test_embeddings_numpy)

#### Replicate model training here

In [ ]:
# create optimizer and loss function
batch_size = 16
lr = 0.05

optimizer = torch.optim.Adam(encoder_model.parameters(), lr=lr)
# optimizer = optim.Adagrad(model.parameters(), lr=lr)
loss_fn = torch.nn.MSELoss()

In [ ]:
# create data loader
dataset_train = torch.utils.data.TensorDataset(test_model_inputs, test_embeddings)
data_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

In [ ]:
# get batch
train_batch, targets_batch = next(iter(data_loader))
print(train_batch.shape)
print(targets_batch.shape)

In [ ]:
from src.data import constants
X = train_batch
encoder_model.to(device=device)

# Compute forward pass
# x_prime = model(train_batch)

# Clear out gradient
encoder_model.zero_grad()

# forward pass
X = X.to(device=device)
batch_size, seq_len = train_batch.size()
print("batch_size", batch_size, "seq_len", seq_len)

# init hidden state before each batch
n_directions = 2 if bidirectional else 1
# hidden = (
#     torch.randn(n_layers * n_directions, batch_size, n_hidden_units).to(device=device),  # initial hidden state
#     torch.randn(n_layers * n_directions, batch_size, n_hidden_units).to(device=device),  # initial cell state
# )

# sort batch by sequence length
# X_lengths = torch.count_nonzero(X, dim=1).to(device="cpu").type(torch.int64)
# ixs = torch.argsort(X_lengths, descending=True)
# X = X[ixs]
# X_lengths = X_lengths[ixs]
# print("X", X.get_device(), "X_lengths", X_lengths.get_device())


eye = torch.eye(constants.VOCAB_SIZE + 1).to(device=device)
X = eye[X]

# pack sequences
# X = pack_padded_sequence(X, X_lengths, batch_first=True, enforce_sorted=True)

# run through LSTM
# all, hidden = encoder_model.lstm(X.to(device), hidden)
all, (hidden, cell) = encoder_model.lstm(X.to(device))
print("hidden", hidden.shape, cell.shape)

embeddings = encoder_model.linear(hidden[0][-1])  # compute the linear model based on the last hidden state of the last layer
print("embeddings", embeddings.shape)

# Compute loss
loss = loss_fn(embeddings, targets_batch.to(encoder_model.device))
# do the backward pass and update parameters
loss.backward()
optimizer.step()

print(loss.item())